In [ ]:
from matplotlib.colors import ListedColormap
from sklearn import preprocessing
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.datasets import make_regression
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import plot_partial_dependence
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import time

### Introduction

In this notebook we will be training a multi-layer regressor on the calfornia housing prices dataset. This is a large dataset with over 20,000 samples. The data comes from the 1990 California census and
summarises housing data by geographical region. We will then see how well our regressor performs on predicting housing prices on an out of training sample.

Being able to carry out a task like this could be important for those looking at potential development projects and the estimated costs or types of houses certain cohorts of people may be interested in.

### Importing Data

In [ ]:
from sklearn.datasets import fetch_california_housing

In [ ]:
a = sklearn.datasets.fetch_california_housing( data_home=None, download_if_missing=True, return_X_y=False, as_frame=True)

In [ ]:
X = a.data
y = a.target
y = y.to_numpy()

## Examining Data

In [ ]:
#First few lines of data
X.head()

In [ ]:
#Histograms of all data features
h = X.hist(bins = 10, figsize = (15,15))

In [ ]:
#Creating heatmap of correlation between housing features
corr = X.corr()
ax = plt.axes()
sns.heatmap(corr, linewidths = 0.5,ax = ax, cmap = 'Spectral')
ax.set_title('Heat map of correlation between data features')
plt.show()

In [ ]:
#Hist of median housing prices
plt.hist(y, bins = 30)
plt.title('Historgram of Median House Prices')
plt.xlabel('Housing price in 100k')
plt.ylabel('# of houses')
plt.show()

### Preparing data

In [ ]:
# Creating traintest split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)

In [ ]:
X_train

In [ ]:
#Scaling feature data using Quantile transformer
quantile_quantformer = QuantileTransformer(n_quantiles=1000)
X_train_quant = quantile_quantformer.fit_transform(X_train)
X_test_quant = quantile_quantformer.transform(X_test)
np.mean(X_test_quant)

In [ ]:
#reshaping to avoid warnings

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

#Transforming target data to zero mean

y_train_scale= y_train - np.mean(y_train)
y_test_scale= y_test - np.mean(y_test)


#Reshape back to avoid warnings
y_train = y_train.ravel()
y_test = y_test.ravel()
y_train_scale = y_train_scale.ravel()
y_test_scale = y_test_scale.ravel()

In [ ]:
#Sanity Check.
regr = MLPRegressor(max_iter=5000, learning_rate_init=0.01,random_state = 7)
scores = cross_val_score(regr,X_train_quant,y_train_scale)  
print("The regressor score for scaled data is", '{0:.4g}'.format(scores.mean()))

### Hyperparameter Tuning

In [ ]:
## Easy way to set hidden layer values for powers of two and ten, can be ignored ##
def Twos(n):
    myList = []
    for j in range(3, n+1):
        number = 2 ** j
        myList.append(number)
    return myList

def alphs(n):
    myList = []
    for j in range(-5, n+1):
        number = 10 ** j
        myList.append(number)
    return myList



In [ ]:
#A single hidden layer with N = 2^3, 2^4,....., 2^7 neurons with the default ’relu’ activation function.

Sizes = Twos(7)

Ncases = len(Sizes)
score_mean = np.zeros(Ncases)
score_std = np.zeros(Ncases)
looptimes = np.zeros(Ncases)

#using loop to train the regressor and then run CV on the regressor and calculate the mean and std of the score
for k in range(Ncases):
    starttime = time.time()
    regr = MLPRegressor(hidden_layer_sizes = (Sizes[k],), random_state=7, activation='relu', 
                        learning_rate_init=0.01, max_iter=5000)
    # This the cross-validation. It is the important and expensive part of the code.
    scores = cross_val_score(regr,X_train_quant,y_train_scale)  
    # record the mean and std of the score
    score_mean[k] = scores.mean()
    score_std[k] = scores.std()
    endtime = time.time()
    looptimes[k] = endtime - starttime
    print("Number of Neurons = ",  (Sizes[k]), ", Avg Score = ",'{0:.4g}'.format(score_mean[k]), 
          ', Time taken for this case is', '{0:.4g}'.format(looptimes[k]))
    
totaltime = sum(looptimes)    

# plot the scores as function of hyperparameter

plt.plot(Sizes,score_mean,'r',label = 'Cross Val Score')
plt.fill_between(Sizes,score_mean-score_std,score_mean+score_std,alpha=0.2,label = 'Score +/- std')
plt.xlabel("N", fontsize=14)
plt.ylabel("mean score +/- std", fontsize=14)
plt.title("Plot of mean scores for varying neuron sizes using Relu Act Func")
plt.legend()
plt.show()    
print("Total training time = ", '{0:.4g}'.format(totaltime), " seconds")

---------
We will now repeat this for a single hidden layer with N = 2^3, 2^4, 2^5, neurons with the ’logistic’ activation function.

In [ ]:

Sizes = Twos(5)
Ncases = len(Sizes)
score_mean = np.zeros(Ncases)
score_std = np.zeros(Ncases)
looptimes = np.zeros(Ncases)
for k in range(Ncases):
    starttime = time.time()
    regr = MLPRegressor(hidden_layer_sizes = (Sizes[k],), random_state=7, activation='logistic', 
                        learning_rate_init=0.01, max_iter=5000)
    # This the cross-validation. It is the important and expensive part of the code.
    scores = cross_val_score(regr,X_train_quant,y_train_scale)  
    # record the mean and std of the score
    score_mean[k] = scores.mean()
    score_std[k] = scores.std()
    endtime = time.time()
    looptimes[k] = endtime - starttime
    print("Number of Neurons = ",  (Sizes[k]), ", Avg Score = ",'{0:.4g}'.format(score_mean[k]), 
          ', Time taken for this case is', '{0:.4g}'.format(looptimes[k]))
totaltime = sum(looptimes)
# plot the scores as function of hyperparameter
plt.plot(Sizes,score_mean,'r',label = 'Cross Val Score')
plt.fill_between(Sizes,score_mean-score_std,score_mean+score_std,alpha=0.2,label = 'Score +/- Std')
plt.xlabel("N", fontsize=14)
plt.ylabel("mean score +/- std", fontsize=14)
plt.title("Plot of mean scores for varying neuron sizes using Logistic Act Func")
plt.legend()
plt.show()    
print("Total training time = ", '{0:.4g}'.format(totaltime), " seconds")

--------
A single hidden layer with N = 32 neurons with ’relu’ activation function, with the regularisation parameter

Alpha = 10^-5, 10^-4,....10^-1

Where alpha is an L2 penalty (regularisation term) parameter.

In [ ]:
#A single hidden layer with N = 32 neurons with ’relu’ activation function, with the regularisation parameter
#alpha = 10^-5, 10^-4,....10^-1

alphas = alphs(-1)
Ncases = len(alphas)
score_mean = np.zeros(Ncases)
score_std = np.zeros(Ncases)
looptimes = np.zeros(Ncases)
for k in range(Ncases):
    starttime = time.time()
    regr = MLPRegressor(alpha=alphas[k],hidden_layer_sizes = (32,), random_state=7, activation='relu', 
                        learning_rate_init=0.01, max_iter=5000)
    # This the cross-validation. It is the important and expensive part of the code.
    scores = cross_val_score(regr,X_train_quant,y_train_scale)  
    # record the mean and std of the score
    score_mean[k] = scores.mean()
    score_std[k] = scores.std()
    endtime = time.time()
    looptimes[k] = endtime - starttime
    print("Alpha = ",  (alphas[k]), ", Avg Score = ",'{0:.4g}'.format(score_mean[k]), 
          ', Time taken for this case is', '{0:.4g}'.format(looptimes[k]))
    
totaltime = sum(looptimes)

# plot the scores as function of hyperparameter
plt.semilogx(alphas,score_mean,'r',label = 'Cross Val Score')
plt.fill_between(alphas,score_mean-score_std,score_mean+score_std,alpha=0.2,label = 'Score +/- std')
plt.xlabel("N", fontsize=14)
plt.ylabel("mean score +/- std", fontsize=14)
plt.title("Plot of mean scores for varying alphas Func")
plt.legend()
plt.show()    
print("Total training time = ", '{0:.4g}'.format(totaltime), " seconds")   

---------
Multiple hidden layers hidden layer sizes = 32, (32,32,32), and (32,32,32,32,32) with the
relu activation function.

In [ ]:
#Multiple hidden layers hidden layer sizes = 32, (32,32,32), and (32,32,32,32,32) with the
#’relu’ activation function.

starttime = time.time()
regr = MLPRegressor(hidden_layer_sizes = (32), random_state=7, activation='relu', 
        learning_rate_init=0.01, max_iter=5000)
# This the cross-validation. It is the important and expensive part of the code.
scores = cross_val_score(regr,X_train_quant,y_train_scale)  
# record the mean and std of the score
score_mean = scores.mean()
score_std = scores.std()
endtime = time.time()
looptimes1 = endtime - starttime

print('layer size of 32 score =','{0:.4g}'.format(score_mean), 'Time taken for this case is', '{0:.4g}'.format(looptimes1))



starttime = time.time()
regr = MLPRegressor(hidden_layer_sizes = (32,32,32), random_state=7, activation='relu', 
        learning_rate_init=0.01, max_iter=5000)
# This the cross-validation. It is the important and expensive part of the code.
scores = cross_val_score(regr,X_train_quant,y_train_scale)  
# record the mean and std of the score
score_mean = scores.mean()
score_std = scores.std()
endtime = time.time()
looptimes2 = endtime - starttime

print('layer size of (32,32,32) score =','{0:.4g}'.format(score_mean), 'Time taken for this case is', '{0:.4g}'.format(looptimes2))



starttime = time.time()
regr = MLPRegressor(hidden_layer_sizes = (32,32,32,32,32), random_state=7, activation='relu', 
        learning_rate_init=0.01, max_iter=5000)
# This the cross-validation. It is the important and expensive part of the code.
scores = cross_val_score(regr,X_train_quant,y_train_scale)  
# record the mean and std of the score
score_mean = scores.mean()
score_std = scores.std()
endtime = time.time()
looptimes3 = endtime - starttime
looptimes = looptimes1 + looptimes2 + looptimes3
print('layer size of (32,32,32,32,32) score =','{0:.4g}'.format(score_mean), 'Time taken for this case is', '{0:.4g}'.format(looptimes3))
      
print("Total training time = ", '{0:.4g}'.format(looptimes), " seconds")

_______________________
##### From the 4 previous test cases it seems logical to set the parameters as follows:

Alpha = 0.0001, for our alpha test using 0.0001 offered the highest mean score and is just as fast as the test cases for 0.001 and 0.01, which offer slightly lower mean cross val scores.

Activation Function = 'relu'. We notice how the relu activation function was typically faster than the logistic activation function with the mean scores being essentially the same, while testing two more values of layer size with relu.

Using three layers with 32 neurons in each also performed the best regarding the hidden layer sizes parameter, while also having a moderately short training time.

In [ ]:
#Testing our final MLP Regressor after our hyper parameter search.


starttime = time.time()
regr = MLPRegressor(alpha = 0.0001, hidden_layer_sizes = (32,32,32), random_state=7, activation='relu', 
        learning_rate_init=0.01, max_iter=5000)
# This the cross-validation. It is the important and expensive part of the code.
scores = cross_val_score(regr,X_train_quant,y_train_scale)  
# record the mean and std of the score
score_mean = scores.mean()
score_std = scores.std()
endtime = time.time()
looptimes = endtime - starttime

print('Our score is','{0:.4g}'.format(score_mean), 'Time taken for this case is', '{0:.4g}'.format(looptimes))
      


In [ ]:
### Trainging and testing on the test data
regr = MLPRegressor(alpha = 0.0001, hidden_layer_sizes = (32,32,32), random_state=7, activation='relu', 
        learning_rate_init=0.01, max_iter=5000)
regr.fit(X_train_quant,y_train_scale)


print("The final regressor score is", '{0:.4g}'.format(regr.score(X_test_quant,y_test_scale)),"\n")

y_predict = regr.predict(X_test_quant)
### Plotting predicted values using our MLP regressor and actual values for same X data to see correspondence 
plt.plot(y_test_scale[:50], '-o', label="y_true")
plt.plot(y_predict[:50],'-o', label="y_predict")
plt.xlabel("Example 0 to 50", fontsize=14)
plt.ylabel("target y", fontsize=14)
plt.title("Predicted vs True values of House Price")
plt.legend()
plt.show()

plt.plot(y_test_scale[1000:1050], '-o', label="y_true")
plt.plot(y_predict[1000:1050],'-o', label="y_predict")
plt.xlabel("Example 1000 to 1050", fontsize=14)
plt.ylabel("target y", fontsize=14)
plt.title("Predicted vs True values of House Price")
plt.legend()


plt.show()

-----------
## Discussion

From our plots and regressor score we can see how we can make extremely good estimations on housing prices having successfully trained a model using a sample from the data and chosen hyperparameters to roughly 80% accuracy. Also given the size of the dataset it also is relatively fast. This could be useful for future building projects in which potential buyers or investors may want to know how lucrative a project may be to undertake. We could improve our scores by trying different train test splits and possibly performing grid searches for the best parameters, but this will be more time consuming and computer intensive

### Partial Dependence Plots

In [ ]:
features = ['MedInc', 'AveOccup', 'HouseAge', 'AveRooms','AveBedrms', 'Population','Latitude','Longitude']
## Need data as a dataframe here hence
X_train_quant_df = pd.DataFrame(data = X_train_quant,index = None, columns = features)

In [ ]:
regr = make_pipeline(MLPRegressor(alpha = 0.01, hidden_layer_sizes = (32,32,32), random_state=7, activation='relu', 
        learning_rate_init=0.01, max_iter=5000))
regr.fit(X_train_quant,y_train_scale)
plot_partial_dependence(regr, X_train_quant_df, features,
                        n_jobs=5, grid_resolution=20)

fig = plt.gcf()
fig.suptitle('Partial dependence of house value on non-location features\n'
             'for the California housing dataset, with MLPRegressor')
fig.subplots_adjust(hspace=0.9)
fig.set_size_inches(9, 9)

From the partial dependence plots we note how house value increases quickly as income increases. We note how houses in less populated areas lead to more expensive houses, likely due to the fact rich areas of california are less densely populated, hence you pay a premium for more "space"
We note how as we decrease longitude and latitude the cheaper houses get.